In [107]:
import pandas as pd
train_pos=pd.read_csv('./data/imdb/train/pos.csv')
train_neg=pd.read_csv('./data/imdb/train/neg.csv')
test_pos=pd.read_csv('./data/imdb/test/pos.csv')
test_neg=pd.read_csv('./data/imdb/test/neg.csv')
print('imported data.....')

def clear_text(text):
    return text.replace('"', '').replace("'", "").replace("<br />","").replace(",","").replace("(","").replace(")","").replace("-","").replace("/","").replace("!","")

train_pos_sent=train_pos['text'].apply(clear_text).tolist()
train_neg_sent=train_neg['text'].apply(clear_text).tolist()
test_pos_sent=test_pos['text'].apply(clear_text).tolist()
test_neg_sent=test_neg['text'].apply(clear_text).tolist()


corpus_train=train_pos_sent+train_neg_sent
corpus_test=test_pos_sent+test_neg_sent


imported data.....


In [33]:
# !pip install nltk
# import nltk
# nltk.download('stopwords')

    100% |################################| 1.4MB 2.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built nltk
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [108]:
from nltk.corpus import stopwords
def create_corpus(corpus):
    corpus_out=[]
    sw=stopwords.words('english')
    for review in corpus:
        review_sents=review.split(".")
        for sent in review_sents:
            temp=sent.split(" ")
            for word in temp:
                if word=='' and word==' ' and word in sw:
                    del word
#             print(sents)
#             print('--------------------------------------')
        corpus_out.append(temp)
    return corpus_out

In [109]:
corpus=create_corpus(corpus_test+corpus_test)

In [49]:
# minl=16000
# maxl=0
# for rev in corpus_train:
#     minl=min(minl,len(rev.split(" ")))
#     maxl=max(maxl,len(rev.split(" ")))
# print(minl,maxl)

(5, 1491)


In [110]:
len(corpus)

50000

In [111]:
from gensim.models import Word2Vec
model = Word2Vec(corpus, min_count=1,seed=123)
print(model)

Word2Vec(vocab=14327, size=100, alpha=0.025)


In [112]:
words = list(model.wv.vocab)
print(words[:10])

['', 'scoreToo', 'BrestLitovsk', 'Savage', 'Bowery', 'foul', 'four', 'Does', 'Olympics', 'spiders']


In [ ]:
model.save('model.bin')
print("model saved")

In [19]:
# from nltk.corpus import stopwords
def create_words(corpus):
    max_word=150
    words=[]
    flag=0
    for review in corpus:
        review_words=[]
        review_sents=review.split(".")
        count=1
        for sent in review_sents:
            temp=sent.split(" ")
            for word in temp:
                if len(word)!=0 and word!=' ' and count<=max_word:
                    review_words.append(word)
                    count+=1
    #         review_words.extend([word for word in sent.split(" ") if word not in stopwords.words('english')])
        words.append(review_words)
    return words

In [20]:
train_words=create_words(corpus_train)

In [21]:
test_words=create_words(corpus_test)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.4834586 , -0.8623024 , -0.60484344, -1.4705721 , -0.12694284,
        2.1140037 ,  0.76141953, -0.22995406,  0.04501231,  0.09383006,
        1.4644676 , -0.7337624 , -1.022652  , -0.5702895 ,  0.98213613,
        0.19339153, -1.157489  ,  2.2790394 ,  0.35115936,  1.0630728 ,
        0.0264265 , -0.5422355 ,  0.2722535 ,  0.17269169, -0.632745  ,
       -0.12009603,  1.8192468 , -0.33960378, -0.86357874, -0.36764932,
        0.04084243,  0.4385397 , -0.8824437 , -0.11205492, -0.25662112,
       -0.38798663,  0.83016354, -0.35403296,  0.2686252 , -1.120308  ,
       -0.14301804, -0.78125286, -1.8001832 ,  1.7826413 , -0.48472777,
        0.1489331 ,  1.1054084 ,  0.51163435, -0.54393524,  1.0574402 ,
        0.26058695,  0.33748442, -0.6892918 ,  0.04279883, -0.33555952,
       -0.11199496,  0.16019207,  1.5494055 , -0.36537984, -1.474116  ,
        0.66464907,  0.12885463,  0.61771953,  0.80684096, -1.7894149 ,
        0.02079906,  0.90201986,  0.04289038,  0.33577064,  1.71

In [ ]:
!pip install gensim

In [13]:
from gensim.models import Word2Vec
new_model = Word2Vec.load('model.bin')

In [18]:
len(train_words[2])

146

In [10]:
import keras

Using TensorFlow backend.


In [59]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense,Conv2D ,LSTM,Lambda
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras import backend as K
tf_session = K.get_session()
import tensorflow as tf

In [62]:
def stack_dim(X):
    conv_outputs=tf.unstack(X,axis=3)
    return tf.concat(conv_outputs,axis=1)

def get_shape(input_shape):
    assert len(input_shape)==4
    return (input_shape[0],input_shape[1]*input_shape[3],input_shape[2])

def model(input_shape,hidden_size,dense_layer1,dense_layer2):
    X_input=Input(input_shape)
    X=Conv2D(32, (5, 5), input_shape=input_shape, activation='relu')(X_input)
    X=Lambda(stack_dim,output_shape=get_shape)(X)
    X=LSTM(hidden_size,return_sequences=True)(X)
    X=Dense(dense_layer1, activation='relu')(X)
    X=Dense(dense_layer2, activation='relu')(X)
    model = Model(inputs = X_input, outputs = X, name='sent_classifier')
    return model

In [63]:
import numpy as np
m=model((300,150,1),128,100,2)

In [65]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 300, 150, 1)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 296, 146, 32)      832       
_________________________________________________________________
lambda_5 (Lambda)            (None, 9472, 146)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 9472, 128)         140800    
_________________________________________________________________
dense_3 (Dense)              (None, 9472, 100)         12900     
_________________________________________________________________
dense_4 (Dense)              (None, 9472, 2)           202       
Total params: 154,734
Trainable params: 154,734
Non-trainable params: 0
_________________________________________________________________
